# Merging ents with tickers

For the sake of autopopulating ORG entities into the app, I use the fact, that the news often contain some sort of a ticker. Those can help us merge recognized organization with its entity in knowledge base without\with few human interference.

Ideally, this should happen in runtime, so I have to design the algorithm in a way to handle all possible cases

In [1]:
%cd ..

/home/codeholder/code/python-playground/app_noisemon


In [2]:
import json
from pathlib import Path
import spacy

/home/master/code/python-playground/app_noisemon/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
input_path = Path("./data/01-filtered/full-time_trading_export_with_ticker.json")
data = json.loads(input_path.read_text())

In [4]:
import regex

In [5]:
nlp = spacy.load("ru_core_news_lg")

In [6]:
data[0]

'$DIS  потенциально может свалиться в бездну, смотрим будет ли прорыв, если будет то тренд вниз на длительное время'

In [7]:
# in the beggining I gonna use only this type of ticker as the least messy
ticker_re = regex.compile("(?<=\$)[A-Z]{1,5}")

In [8]:
data = list(filter(lambda text: any(ticker_re.findall(text)), data))

In [9]:
len(data)

44916

In [10]:
data[2]

'$WB   Отличная инвест идея подсказанная участником нашей группы! Отлично смотрится в Лонг (от 46.2) при хорошем отчёте о занятости американцев и твиттам Трампа не содержащих слов "война", "война с Китаем", "главный враг Америки Xi" По ТА и по индикатором цена хочет выйти из красной зоны как было с  $NFLX \n\nПотенциальное сопротивление 47 48 49, пробивая 51.40 летим вообще отлично в горизонте месяца'

In [11]:
from noisemon.data_processing.wikidata import Wikidata

In [13]:
wikidata = Wikidata()

In [15]:
wikidata.lookup_aliases_by_ticker("BABA")

defaultdict(set,
            {'Q1359568': {'ALIBABA GROUP',
              'ALIBABA GROUP HOLDING',
              'Alibaba',
              'Alibaba Group',
              'Alibaba Group (Cayman Islands)',
              'Alibaba Group Holding Limited',
              'Alibaba, Алибаба, Alibaba Group Holding Limited',
              'Alibaba.com',
              'Grupo Alibaba',
              'Kumpulan Alibaba',
              '阿里巴巴',
              '阿里巴巴公司',
              '阿里巴巴集团',
              '阿里巴巴集团控股有限公司',
              '阿里巴巴集団',
              '阿里巴巴集団控股',
              '阿里巴巴集團',
              '阿里巴巴集團控股',
              '阿里巴巴集團控股有限公司'}})

In [27]:
def process(text):
    doc = nlp(text)
    entities = [entity.text for entity in doc.ents if entity.label_ =="ORG"]
    print(f"Detected Entities: {entities}")
    if not entities:
        return
    
    tickers = set(ticker_re.findall(text))
    print(f"Detected Tickers: {tickers}")
    # We do not know, which ticker belongs to which ORG at all,
    # So it's reasonable to perform lookup through all aliases
    possible_orgs = {} # QID : {set of aliases}
    for ticker in tickers:
        lookup_result = wikidata.lookup_aliases_by_ticker(ticker)
        possible_orgs.update(lookup_result)
    # this can erase some entities with overlapping aliases, but let it be so for now
    reverse_index = {alias.lower(): qid for qid, alias_set in possible_orgs.items() for alias in alias_set}
    
    organizations_matched = []
    organizations_mismatched = []
    #TODO similarity calctulation
    for organization in entities:
        QID = reverse_index.get(organization.lower(), None)
        if QID:
            organizations_matched.append((organization, QID))
        else:
            organizations_mismatched.append(organization)
            
    print(f"Matched: {organizations_matched}")
    print(f"Mismatched: {organizations_mismatched}")


In [ ]:
i=7

In [41]:
i += 1
print(data[i])
process(data[i])

⁠ 👍 ГИГАНТЫ ПЕРЕЕЗЖАЮТ —  APPLE, MICROSOFT, GOOGLE ДУМАЮТ О ПЕРЕНОСЕ СВОИХ ПРОИЗВОДСТВ ИЗ КИТАЯ 
В связи с торговой войной между США и Китаем в прошлом году и появлением нового коронавируса Apple  $AAPL  , Microsoft  $MSFT  и Google  $GOOG  решили вывести большие части своих производств.
По слухам из достоверного источника Google собирается начать производство нового недорогого смартфона, который, по слухам, будет называться Pixel 4a, во Вьетнаме уже в апреле.
Между тем, Microsoft надеется начать производство во Вьетнаме во втором квартале для своей линейки ноутбуков и настольных ПК Surface.
В прошлом году Apple, по сообщениям, пыталась начать испытания производства своих AirPods во Вьетнаме и просила поставщиков изучить возможность перемещения от 15% до 30% производства из Китая в другие части Юго-Восточной Азии. ( CNBC )
Detected Entities: ['MICROSOFT', 'GOOGLE ДУМАЮТ О', 'Apple', 'Microsoft', 'Google', 'Google', 'Microsoft', 'Apple']
Detected Tickers: {'AAPL', 'MSFT', 'GOOG'}
Matche

In [24]:
data[2]

'https://youtu.be/m-wBa1RsUJ0 \n\n⚠️Добавьте себе в watch list  $TSLA \nЦена болтается в симметричном треугольнике \n\nЕсли она вылетит из него она вылетит на высоту этого клина\n\nНа видео вылет показан наверх\n⚠️ 💡В реальности он может быть в любую сторону. Как только цена пересекаются границу вырастет моментум и образуется длинная свеча прорыва. Это означает что паттерн сломан и направление задано\n\nВсе играют этот прорыв, когда видят что цена пересекает границу и несётся вниз или наверх без всяких "или"'